In [1]:
import torch
from diffusers import FluxPipeline

In [2]:
import sentencepiece

In [3]:
# 31G, 140m
# pipe = FluxPipeline.from_pretrained('black-forest-labs/FLUX.1-dev', torch_dtype=torch.bfloat16)

In [4]:
# 无法运行
# MBP ~150H 10630.34s/it
# prompt = "A cat holding a sign that says 你好 world"
# image = pipe(
#     prompt,
#     height=1024,
#     width=1024,
#     guidance_scale=3.5,
#     num_inference_steps=50,
#     max_sequence_length=512,
#     generator=torch.Generator("cpu").manual_seed(0)
# ).images[0]